In [1]:
pip install xFormers

In [ ]:
pip install  diffusers

In [ ]:
pip install gradio

In [3]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image
from PIL import Image
import gradio as gr
try:
    pipeline = AutoPipelineForImage2Image.from_pretrained(
        "stabilityai/stable-diffusion-xl-refiner-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

tokenizer_2/tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/4.52G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Model loaded successfully!


In [4]:
pipeline.enable_model_cpu_offload()


In [5]:
pipeline.enable_xformers_memory_efficient_attention()


In [9]:
def transform_sari(image, prompt, strength):
    try:
        # Load and convert the image
        print("Loading image...")
        init_image = Image.open(image).convert("RGB")

        # Run the pipeline for the transformation
        print("Generating image with prompt:", prompt)
        output_image = pipeline(prompt, image=init_image, strength=strength).images[0]

        print("Image generated successfully!")
        return output_image
    except Exception as e:
        print(f"Error during image generation: {e}")
        return None

# Set up the Gradio interface
interface = gr.Interface(
    fn=transform_sari,  # The function to call
    inputs=[
        gr.Image(type="filepath", label="Upload Initial Image"),  # Input image
        gr.Textbox(lines=2, placeholder="Enter a transformation prompt", label="Prompt"),  # Text input for prompt
        gr.Slider(0.0, 1.0, value=0.95, label="Strength")  # Slider for strength adjustment
    ],
    outputs=gr.Image(label="Transformed Image"),  # Display the output image
    title="Sari Transformation",
    description="Upload an image of a sari and provide a prompt to transform it (e.g., 'Convert the sari to vibrant red'). Adjust the strength with the slider."
)

In [10]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://979d8c873795aee285.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
